## Deliverable 2. Create a Customer Travel Destinations Map.

In [89]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [90]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,College,US,64.8569,-147.8028,-2.25,77,75,0.00,light snow
1,1,Bandarbeyla,SO,9.4942,50.8122,75.31,62,0,14.18,clear sky
2,2,Bilibino,RU,68.0546,166.4372,-29.29,100,94,3.06,overcast clouds
3,3,Beringovskiy,RU,63.0500,179.3167,6.01,88,100,13.98,overcast clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,72.99,82,76,10.00,broken clouds


In [109]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [110]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bandarbeyla,SO,9.4942,50.8122,75.31,62,0,14.18,clear sky
4,4,Saint-Philippe,RE,-21.3585,55.7679,72.99,82,76,10.00,broken clouds
5,5,Rikitea,PF,-23.1203,-134.9692,76.44,73,0,17.34,clear sky
11,11,Cidreira,BR,-30.1811,-50.2056,65.89,70,29,7.83,scattered clouds
13,13,Mataura,NZ,-46.1927,168.8643,68.49,37,52,0.85,light rain
...,...,...,...,...,...,...,...,...,...,...
621,621,Shahpur,IN,16.7000,76.8333,68.13,92,14,9.01,few clouds
637,637,Nsanje,MW,-16.9200,35.2620,74.98,90,19,4.09,few clouds
641,641,Taoudenni,ML,22.6783,-3.9836,67.51,15,100,11.70,overcast clouds
647,647,Doha,QA,25.2867,51.5333,71.58,83,11,4.61,few clouds


In [111]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [112]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bandarbeyla,SO,9.4942,50.8122,75.31,62,0,14.18,clear sky
4,4,Saint-Philippe,RE,-21.3585,55.7679,72.99,82,76,10.00,broken clouds
5,5,Rikitea,PF,-23.1203,-134.9692,76.44,73,0,17.34,clear sky
11,11,Cidreira,BR,-30.1811,-50.2056,65.89,70,29,7.83,scattered clouds
13,13,Mataura,NZ,-46.1927,168.8643,68.49,37,52,0.85,light rain
...,...,...,...,...,...,...,...,...,...,...
621,621,Shahpur,IN,16.7000,76.8333,68.13,92,14,9.01,few clouds
637,637,Nsanje,MW,-16.9200,35.2620,74.98,90,19,4.09,few clouds
641,641,Taoudenni,ML,22.6783,-3.9836,67.51,15,100,11.70,overcast clouds
647,647,Doha,QA,25.2867,51.5333,71.58,83,11,4.61,few clouds


In [113]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [114]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [115]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location']=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [116]:
# 7. Drop the rows where there is no Hotel Name.
len(hotel_df[hotel_df['Hotel Name'] == ''])

19

In [127]:
#delete_hotels = hotel_df[hotel_df['Hotel Name']==''].index
delete_hotels = hotel_df[hotel_df['Hotel Name']==''].index
hotel_df.drop(index=delete_hotels,inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bandarbeyla,SO,75.31,clear sky,9.4942,50.8122,JABIR HOTEL
4,Saint-Philippe,RE,72.99,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
5,Rikitea,PF,76.44,clear sky,-23.1203,-134.9692,People ThankYou
11,Cidreira,BR,65.89,scattered clouds,-30.1811,-50.2056,Hotel Castelo
13,Mataura,NZ,68.49,light rain,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
620,Kasongo,CD,67.33,overcast clouds,-4.4500,26.6667,Guest Hôtel
621,Shahpur,IN,68.13,few clouds,16.7000,76.8333,Ashwini Hotel & Deluxe Lodge
637,Nsanje,MW,74.98,few clouds,-16.9200,35.2620,Nsanje Discovery Lodge
647,Doha,QA,71.58,few clouds,25.2867,51.5333,Movenpick Hotel Doha


In [129]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [132]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template ="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [133]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0),zoom_level = 1.6)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))